In [3]:
import pandas as pd
train_data = pd.read_parquet('../data/XGBoost_data.parquet')
train_data = train_data.sort_values(by=['symbol_id', 'date_id', 'time_id'])


In [4]:
# List of responder columns
responder_columns = [col for col in train_data.columns if col.startswith('responder_')]

# Generate lag features
for responder in responder_columns:
    train_data[f'{responder}_lag_1'] = train_data.groupby('symbol_id')[responder].shift(1)


In [5]:
for responder in responder_columns:
    train_data[f'{responder}_rolling_mean_3'] = train_data.groupby('symbol_id')[responder].rolling(window=3).mean().reset_index(0, drop=True)
for responder in responder_columns:
    train_data[f'{responder}_lag_diff_1'] = train_data[responder] - train_data[f'{responder}_lag_1']
    train_data[f'{responder}_lag_ratio_1'] = train_data[responder] / (train_data[f'{responder}_lag_1'] + 1e-8)


In [ ]:
print(train_data.head())
print(train_data.columns)


In [9]:
train_data.fillna(0, inplace=True)


In [ ]:
train_data.to_parquet('../data/XGBoost_data.parquet', index=False)

In [ ]:
import pandas as pd
updated_train_data = pd.read_parquet('../data/XGBoost_data.parquet')
print(updated_train_data.head())
print(updated_train_data.columns)


In [1]:
import pandas as pd
train_data = pd.read_parquet('../data/XGBoost_data.parquet')


# Identify feature columns (e.g., feature_00, feature_01, ...)
feature_cols = [col for col in train_data.columns if col.startswith('feature_')]

# Optional: Verify responder columns remain untouched
responder_cols = [col for col in train_data.columns if col.startswith('responder_')]

print(f"Feature Columns: {len(feature_cols)}")
print(f"Responder Columns: {len(responder_cols)}")


Feature Columns: 67
Responder Columns: 45


In [2]:
from sklearn.feature_selection import VarianceThreshold

# Apply VarianceThreshold on feature columns
selector = VarianceThreshold(threshold=0.01)  # 1% variance threshold
filtered_features = selector.fit_transform(train_data[feature_cols])

# Get remaining feature names
selected_feature_names = selector.get_feature_names_out(feature_cols)

# Create a new DataFrame with filtered features and untouched responders
train_data_filtered = pd.concat([
    pd.DataFrame(filtered_features, columns=selected_feature_names),  # Filtered features
    train_data[responder_cols],  # Original responders
    train_data[['date_id', 'time_id', 'symbol_id']]  # Other key columns
], axis=1)


In [3]:
train_data_filtered.to_parquet('../data/XGBoost_data_filtered.parquet', index=False)


In [1]:
import pandas as pd
updated_train_data = pd.read_parquet('../data/XGBoost_data_filtered.parquet')
print(updated_train_data.head())
print(updated_train_data.columns)

   feature_00  feature_01  feature_04  feature_05  feature_06  feature_07  \
0    3.674681    0.559137   -2.866148    1.045130    1.190328   -0.050570   
1    3.655818   -0.729315   -2.866148    1.463785    1.666280    0.389528   
2    3.674681   -0.448754   -2.866148    1.694624    1.666280    1.319947   
3    3.461334   -0.521957   -2.547946    1.694624    1.666280    1.692438   
4    3.524649   -0.812735   -2.866148    1.694624    1.666280    1.692438   

   feature_08  feature_09  feature_10  feature_11  ...  \
0   -0.294838        11.0         7.0        76.0  ...   
1   -0.634664        11.0         7.0        76.0  ...   
2   -0.586630        11.0         7.0        76.0  ...   
3   -0.637521        11.0         7.0        76.0  ...   
4   -0.627068        11.0         7.0        76.0  ...   

   responder_5_lag_ratio_1  responder_6_lag_diff_1  responder_6_lag_ratio_1  \
0                 0.000000                0.000000                 0.000000   
1                 0.024018    